In [ ]:
import tensorflow as tf
import os
import matplotlib.pyplot as plt

# === CONFIGURATION ===
IMG_HEIGHT = 256
IMG_WIDTH = 256
BATCH_SIZE = 1  # CycleGAN typically uses 1 image per batch
AUTOTUNE = tf.data.AUTOTUNE

# === PATHS ===
trainA_path = "/content/dataset/dataset/trainA"
trainB_path = "/content/dataset/dataset/trainB"


# === IMAGE LOADING FUNCTION ===
def load_image(file_path):
    """Reads and decodes image file."""
    img = tf.io.read_file(file_path)
    img = tf.image.decode_png(img, channels=3)
    return tf.image.convert_image_dtype(img, tf.float32)  # converts to [0, 1]

# === PREPROCESSING FUNCTION ===
def preprocess_image(img):
    """Resizes and normalizes the image to [-1, 1]."""
    img = tf.image.resize(img, [IMG_HEIGHT, IMG_WIDTH])
    img = (img - 0.5) * 2  # normalize from [0,1] → [-1,1]
    return img

# === LOAD FILE PATHS ===
trainA_files = tf.data.Dataset.list_files(os.path.join(trainA_path, "*.png"), shuffle=True)
trainB_files = tf.data.Dataset.list_files(os.path.join(trainB_path, "*.png"), shuffle=True)

# === MAP FUNCTIONS ===
trainA_dataset = (trainA_files
                  .map(load_image, num_parallel_calls=AUTOTUNE)
                  .map(preprocess_image, num_parallel_calls=AUTOTUNE)
                  .shuffle(100)
                  .batch(BATCH_SIZE))

trainB_dataset = (trainB_files
                  .map(load_image, num_parallel_calls=AUTOTUNE)
                  .map(preprocess_image, num_parallel_calls=AUTOTUNE)
                  .shuffle(100)
                  .batch(BATCH_SIZE))

# === VERIFY A FEW SAMPLES ===
sample_ct = next(iter(trainA_dataset))
sample_pet = next(iter(trainB_dataset))

def denormalize(img):
    """Convert [-1,1] → [0,1] for display."""
    return (img + 1) / 2

plt.figure(figsize=(8, 4))
plt.subplot(1, 2, 1)
plt.title("CT Sample (trainA)")
plt.imshow(denormalize(sample_ct[0]))
plt.axis("off")

plt.subplot(1, 2, 2)
plt.title("PET Sample (trainB)")
plt.imshow(denormalize(sample_pet[0]))
plt.axis("off")
plt.show()